In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Concatenate, GlobalAveragePooling2D

In [4]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np

In [5]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import os
import matplotlib.pyplot as plt
import soundfile as sf
import numpy as np
from itertools import chain
import pandas as pd
import numpy
from sklearn import metrics
import librosa.display as dsp
import librosa
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
import soundfile as sf
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import wave
from scipy.io import wavfile
import pylab
import itertools
from pathlib import Path

In [6]:
!pip install --quiet wfdb 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 KB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
librosa 0.10.0.post2 requires soundfile>=0.12.1, but you have soundfile 0.11.0 which is incompatible.


In [7]:
import wfdb

In [8]:
base_dir = "/content/drive/MyDrive/Parkinson_LONDON"

read_dir = os.path.join(base_dir, 'ReadText')
dialogue_dir = os.path.join(base_dir, 'SpontaneousDialogue')


saudavels_dir = os.path.join(read_dir, 'HC')  # diretório com as imagens de treinamento da classe saudavel
patologicos_dir = os.path.join(read_dir, 'PD')  # diretório com as imagens de treinamento da classe patologico 
read_todos_dir = saudavels_dir+patologicos_dir
# validation_saudavels_dir = os.path.join(dialogue_dir, 'HC')  # diretório com as imagens de validação da classe saudavel 
# validation_patologicos_dir = os.path.join(dialogue_dir, 'PD')  # diretório com as imagens de validação da classe patologico

In [9]:
num_HC_tr = len(os.listdir(saudavels_dir))
num_patologicos_tr = len(os.listdir(patologicos_dir))

# num_saudavels_val = len(os.listdir(validation_saudavels_dir))
# num_patologicos_val = len(os.listdir(validation_patologicos_dir))

total_train = num_HC_tr + num_patologicos_tr
# total_val = num_saudavels_val + num_patologicos_val
# total_total = total_train + total_val

print('total de "imagens" de saudavels na base:', num_HC_tr)
print('total de "imagens" de patologicos na base:', num_patologicos_tr)

# print('total de "imagens" de saudavels na base de validação:', num_saudavels_val)
# print('total de "imagens" de patologicos na base de validacao:', num_patologicos_val)
print("---")
print("Total de 'imagens' de treino:", total_train)
# print("Total de 'imagens' de validação:", total_val)
# print("Total de 'imagens':", total_total)

total de "imagens" de saudavels na base: 21
total de "imagens" de patologicos na base: 16
---
Total de 'imagens' de treino: 37


In [12]:
l_train_saudaveis = sorted(list(set([x[0:8] for x in os.listdir(saudavels_dir)])))


l_sinais_train_saudaveis = []
  
l_train_saudaveis_wav = [x for x in os.listdir(saudavels_dir) if x.endswith('.wav')]

In [13]:
l_train_saudaveis_wav

['0ID00_hc_0_0_0.wav',
 '0ID09_hc_0_0_0.wav',
 '0ID10_hc_0_0_0.wav',
 '0ID01_hc_0_0_0.wav',
 '0ID03_hc_0_0_0.wav',
 '0ID08_hc_0_0_0.wav',
 '0ID05_hc_0_0_0.wav',
 '0ID35_hc_0_0_0.wav',
 '0ID21_hc_0_0_0.wav',
 '0ID36_hc_0_0_0.wav',
 '0ID15_hc_0_0_0.wav',
 '0ID31_hc_0_1_1.wav',
 '0ID14_hc_0_0_0.wav',
 '0ID12_hc_0_0_0.wav',
 '0ID11_hc_0_0_0.wav',
 '0ID28_hc_0_0_0.wav',
 '0ID19_hc_0_0_0.wav',
 '0ID25_hc_0_0_0.wav',
 '0ID26_hc_0_0_0.wav',
 '0ID22_hc_0_0_0.wav',
 '0ID23_hc_0_0_0.wav']

In [16]:
l_train_patologico = sorted(list(set([x[0:8] for x in os.listdir(patologicos_dir)])))
#l_train_patologico
  
l_train_patologico_wav = [x for x in os.listdir(patologicos_dir) if x.endswith('.wav')]

In [17]:
l_train_patologico_wav

['1ID07_pd_2_0_0.wav',
 '1ID33_pd_3_2_2.wav',
 '1ID32_pd_3_1_1.wav',
 '1ID30_pd_2_1_1.wav',
 '1ID27_pd_4_1_1.wav',
 '1ID13_pd_3_2_2.wav',
 '1ID06_pd_3_1_1.wav',
 '1ID29_pd_3_1_2.wav',
 '1ID02_pd_2_0_0.wav',
 '1ID17_pd_2_1_0.wav',
 '1ID34_pd_2_0_0.wav',
 '1.wav',
 '1ID20_pd_3_0_1.wav',
 '1ID16_pd_2_0_0.wav',
 '1ID24_pd_2_0_0.wav',
 '1ID18_pd_4_3_3.wav']

In [18]:
l_total = l_train_saudaveis_wav+l_train_patologico_wav

In [19]:
wav_data2 = []
for audio_file in l_train_saudaveis_wav:
  audio_file = '/content/drive/MyDrive/Parkinson_LONDON/ReadText/HC/' + audio_file   
      #read wav data
  audio, sr = librosa.load(audio_file, sr= 8000, mono=True)
  clips = librosa.effects.split(audio, top_db=20)
  wav_data = []
  for c in clips:
    print(c)
    data = audio[c[0]: c[1]]
    wav_data.extend(data)
    wav_data2.append(wav_data)
    sf.write(audio_file, wav_data, sr)
#print(audio.shape, sr)

[     0 871424]
[   512 637952]
[     0 732672]
[     0 667136]
[     0 781312]
[     0 691200]
[     0 605696]
[     0 433152]
[     0 641536]
[     0 762880]
[     0 973824]
[      0 1024512]
[     0 879886]
[     0 703488]
[     0 734720]
[     0 813056]
[  1024 770560]
[     0 734720]
[     0 777728]
[     0 798720]
[    512 1044480]


In [28]:
lista_sinais_saudavel = []
for audio_file in wav_data2:
  data = np.array(audio_file).reshape(len(audio_file), 1)
  lista_sinais_saudavel.append(data)

In [31]:
wav_data3 = []
for audio_file in l_train_patologico_wav:
  audio_file = '/content/drive/MyDrive/Parkinson_LONDON/ReadText/PD/' + audio_file   
      #read wav data
  audio, sr = librosa.load(audio_file, sr= 8000, mono=True)
  clips = librosa.effects.split(audio, top_db=20)
  wav_data = []
  for c in clips:
    print(c)
    data = audio[c[0]: c[1]]
    wav_data.extend(data)
    wav_data3.append(wav_data)
    sf.write(audio_file, wav_data, sr)
    #wav_data3.append(n)
#print(audio.shape, sr)

[     0 735744]
[     0 367616]
[     0 346112]
[     0 719360]
[  1024 380416]
[   512 327564]
[     0 887808]
[   512 565760]
[     0 177664]
[     0 524800]
[     0 747520]
[     0 639891]
[     0 611840]
[     0 839168]
[     0 729600]
[     0 310272]


In [57]:
l_total = wav_data2 + wav_data3

In [32]:
lista_sinais_patologico = []
for audio_file in wav_data3:
  data = np.array(audio_file).reshape(len(audio_file), 1)
  lista_sinais_patologico.append(data)

In [47]:
for i in lista_sinais_patologico:
  print(i.shape)

(735744, 1)
(367616, 1)
(346112, 1)
(719360, 1)
(379392, 1)
(327052, 1)
(887808, 1)
(565248, 1)
(177664, 1)
(524800, 1)
(747520, 1)
(639891, 1)
(611840, 1)
(839168, 1)
(729600, 1)
(310272, 1)


In [48]:
for i in lista_sinais_saudavel:
  print(i.shape)

(871424, 1)
(637440, 1)
(732672, 1)
(667136, 1)
(781312, 1)
(691200, 1)
(605696, 1)
(433152, 1)
(641536, 1)
(762880, 1)
(973824, 1)
(1024512, 1)
(879886, 1)
(703488, 1)
(734720, 1)
(813056, 1)
(769536, 1)
(734720, 1)
(777728, 1)
(798720, 1)
(1043968, 1)


In [50]:
lista_sinais_totais = lista_sinais_saudavel + lista_sinais_patologico

In [65]:
array_y_train = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

In [66]:
array_y_train = np.array(array_y_train)

## Transformando em Imagens

In [ ]:
INPUT_DIR = "/content/drive/MyDrive/Parkinson_LONDON/frases@"
OUTPUT_DIR = "/content/drive/MyDrive/kaggle2"

# Print names of 10 WAV files from the input path
parent_list = os.listdir(INPUT_DIR)
for i in range(10):
    print(parent_list[i])

0ID00_hc_0_0_0.wav
0ID01_hc_0_0_0.wav
1ID34_pd_2_0_0.wav
1ID33_pd_3_2_2.wav
1ID32_pd_3_1_1.wav
1ID30_pd_2_1_1.wav
1ID29_pd_3_1_2.wav
1ID27_pd_4_1_1.wav
1ID24_pd_2_0_0.wav
1ID20_pd_3_0_1.wav


In [ ]:
# Utility function to get sound and frame rate info
def get_wav_info(wav_file):     ##aqui é uma função com as informações pra trnasformar a wav em espectograma
    wav = wave.open(wav_file, 'r')
    frames = wav.readframes(-1)
    sound_info = pylab.frombuffer(frames, 'int16')
    frame_rate = wav.getframerate()
    wav.close()
    return sound_info, frame_rate

# For every recording, make a spectogram and save it as label_speaker_no.png
if not os.path.exists(os.path.join(OUTPUT_DIR, 'audio-images')):                    ##se nao existir uma pasta chamada audio-images ele deve criar uma
    os.mkdir(os.path.join(OUTPUT_DIR, 'audio-images'))
    
for filename in os.listdir(INPUT_DIR):                                                    ##criou-se um for para passar por todos os arquivos .wav
    if "wav" in filename:                                                                 ##se terminar com wav o arquivo entra nas instruções
        file_path = os.path.join(INPUT_DIR, filename)                                     ##aqui o os.path.join cria um caminho e coloca na variavel file_path
        file_stem = Path(file_path).stem                                                  ##aqui atraves do stem ele tira o sufixo pra ficar so com a palavra
        target_dir = f'class_{file_stem[0]}'                                              ##aqui ele cria uma classe com o primeiro sufixo que tiver e coloca em target_dir
        dist_dir = os.path.join(os.path.join(OUTPUT_DIR, 'audio-images'), target_dir)     ##
        file_dist_path = os.path.join(dist_dir, file_stem)
        if not os.path.exists(file_dist_path + '.png'):
            if not os.path.exists(dist_dir):
                os.mkdir(dist_dir)
            file_stem = Path(file_path).stem
            sound_info, frame_rate = get_wav_info(file_path)
            pylab.specgram(sound_info, Fs=frame_rate)
            pylab.savefig(f'{file_dist_path}.png')
            pylab.close()

# Print the ten classes in our dataset
path_list = os.listdir(os.path.join(OUTPUT_DIR, 'audio-images'))
print("Classes: \n")
for i in range(2):
    print(path_list[i])
    
# File names for class 0
path_list = os.listdir(os.path.join(OUTPUT_DIR, 'audio-images/class_0'))
print("\nAlguns arquivos png: \n")
for i in range(10):
    print(path_list[i])

Classes: 

class_0
class_1

Alguns arquivos png: 

0ID36_hc_0_0_0.png
0ID35_hc_0_0_0.png
0ID31_hc_0_1_1.png
0ID28_hc_0_0_0.png
0ID26_hc_0_0_0.png
0ID25_hc_0_0_0.png
0ID23_hc_0_0_0.png
0ID22_hc_0_0_0.png
0ID21_hc_0_0_0.png
0ID19_hc_0_0_0.png


##ResNet e Modelo

In [33]:
# Load ResNet50 model
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 [==============================] - 0s 0us/step


In [34]:
# Freeze layers in ResNet50 model
for layer in resnet_model.layers:
    layer.trainable = False

In [35]:
# Cut model before Flatten layer and add new Dense layer
output_resnet = resnet_model.layers[-2].output
# new_model = Dense(128, activation='relu')(output_resnet)
# new_model = Model(inputs=resnet_model.input, outputs=new_model)

In [ ]:
# Compile new model
new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# path_list2 = os.listdir(os.path.join(OUTPUT_DIR, 'audio-images/class_0'))
# path_list2

In [ ]:
# path2 = "/content/drive/MyDrive/kaggle2/audio-images/class_0/"

In [ ]:
# batch_list = []

# for nome in path_list2:
#   img_path = path2 + nome
#   img = image.load_img(img_path, target_size=(224, 224))
#   x = image.img_to_array(img)
#   x = np.expand_dims(x, axis=0)
#   x = preprocess_input(x)
#   batch_list.append(x)

# batch = np.vstack(batch_list) #transforma em uma única matriz
# output = resnet_model.predict(batch)


1/1 [==============================] - 5s 5s/step


In [ ]:
output.shape

(21, 7, 7, 2048)

#REDE PRE TREINADA


In [41]:
model_pa = "/content/drive/MyDrive/Colab Notebooks/wavenet_pre_treined.h5"

In [42]:
model_A = tf.keras.models.load_model(model_pa)

In [43]:
model_pre_train = tf.keras.models.Model(inputs=model_A.input, outputs=model_A.layers[-3].output)

In [44]:
x = tf.keras.layers.Dense(1, activation = 'sigmoid')(model_pre_train.output)    #saida igual a 1
model_pre_train = tf.keras.models.Model(inputs=model_pre_train.input, outputs=x)

In [45]:
for i in model_pre_train.layers[:-2]:
  # print(i)
  i.trainable = False

In [46]:

opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
model_pre_train.compile(loss='binary_crossentropy', optimizer=opt,  metrics = ['accuracy'])

In [ ]:
def build_model(summary=False):

  model = models.Sequential()
  model.add(layers.Conv1D(32, 5, activation='relu', input_shape = (825, 1)))
  model.add(layers.MaxPool1D())
  model.add(layers.Conv1D(64, 5, activation='relu')) # 64 feature maps
  model.add(layers.MaxPool1D())
  model.add(layers.Flatten())
  model.add(layers.Dense(100, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))


  if summary:
    model.summary()

  model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

  return model

model = build_model(summary=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 821, 32)           192       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 410, 32)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 406, 64)           10304     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 203, 64)          0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 12992)             0         
                                                                 
 dense_1 (Dense)             (None, 100)               1

In [67]:
from sklearn.model_selection import train_test_split
X_val, X_test, y_val, y_test = train_test_split(lista_sinais_totais, array_y_train, random_state=42, test_size = 0.5)

In [68]:
history = model_pre_train.fit(X_val,
                    y_val,
                    epochs = 1,  
                    batch_size = 32,
                    validation_data =[X_test,y_test])

ValueError: ignored